In [37]:
import pandas as pd
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from datetime import timezone, datetime
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
#from sklearn.compose import ColumnTransformer 
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold 
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import classification_report, confusion_matrix

import itertools

In [6]:
from julia import Julia

ModuleNotFoundError: No module named 'julia'

In [5]:
%load_ext julia.magic

ModuleNotFoundError: No module named 'julia'

In [4]:
from pycall import X4

ModuleNotFoundError: No module named 'pycall'

In [1]:
X4

NameError: name 'X4' is not defined

# Importing Dataset

In [38]:
df_pat = pd.read_csv("../data/SPARCS_kidney_cancer_binary.csv")
quad4 = pd.read_csv("../data/cancer4quad.csv")
diff4 =  pd.read_csv("../data/cancer4diff.csv")

In [39]:
df_pat.head()

,Unnamed: 0,Total_Charges,APR_Severity_of_Illness_Code,APR_Risk_of_Mortality,Age_Ordinal,Length_of_Stay,Health_Service_Area_New York City,Hospital_County_Bronx,Hospital_County_Kings,Hospital_County_Manhattan,Hospital_County_Queens,Hospital_County_Richmond,Type_of_Admission_Elective,Type_of_Admission_Emergency,Type_of_Admission_Urgent,Type_of_Admission_Not Available,Patient_Disposition_Home or Self Care,Patient_Disposition_Medicare Cert Long Term Care Hospital,Patient_Disposition_Home w/ Home Health Services,Patient_Disposition_Expired,Patient_Disposition_Skilled Nursing Home,Patient_Disposition_Short-term Hospital,Patient_Disposition_Hospice - Medical Facility,Patient_Disposition_Facility w/ Custodial/Supportive Care,Patient_Disposition_Left Against Medical Advice,Patient_Disposition_Inpatient Rehabilitation Facility,Patient_Disposition_Hospice - Home,Patient_Disposition_Psychiatric Hospital or Unit of Hosp,Patient_Disposition_Cancer Center or Children's Hospital,CCS_Diagnosis_Code_33,CCS_Procedure_Code_104,CCS_Procedure_Code_110,CCS_Procedure_Code_83,CCS_Procedure_Code_196,CCS_Procedure_Code_90,CCS_Procedure_Code_175,CCS_Procedure_Code_67,CCS_Procedure_Code_231,CCS_Procedure_Code_0,CCS_Procedure_Code_198,CCS_Procedure_Code_213,CCS_Procedure_Code_34,CCS_Procedure_Code_169,CCS_Procedure_Code_99,CCS_Procedure_Code_217,CCS_Procedure_Code_88,CCS_Procedure_Code_54,CCS_Procedure_Code_222,CCS_Procedure_Code_193,CCS_Procedure_Code_179,CCS_Procedure_Code_97,CCS_Procedure_Code_73,CCS_Procedure_Code_202,CCS_Procedure_Code_211,CCS_Procedure_Code_112,CCS_Procedure_Code_40,CCS_Procedure_Code_61,CCS_Procedure_Code_118,CCS_Procedure_Code_173,CCS_Procedure_Code_227,CCS_Procedure_Code_102,CCS_Procedure_Code_205,CCS_Procedure_Code_201,CCS_Procedure_Code_58,CCS_Procedure_Code_226,CCS_Procedure_Code_101,CCS_Procedure_Code_95,CCS_Procedure_Code_100,CCS_Procedure_Code_9,CCS_Procedure_Code_87,CCS_Procedure_Code_224,CCS_Procedure_Code_78,CCS_Procedure_Code_37,CCS_Procedure_Code_191,CCS_Procedure_Code_103,CCS_Procedure_Code_57,CCS_Procedure_Code_64,CCS_Procedure_Code_39,CCS_Procedure_Code_197,CCS_Procedure_Code_209,CCS_Procedure_Code_113,CCS_Procedure_Code_70,CCS_Procedure_Code_158,CCS_Procedure_Code_216,CCS_Procedure_Code_187,APR_DRG_Code_442,APR_DRG_Code_461,APR_DRG_Code_447,APR_DRG_Code_441,APR_DRG_Code_4,APR_DRG_Code_951,APR_DRG_Code_952,APR_DRG_Code_446,APR_DRG_Code_950,APR_DRG_Code_3,APR_MDC_Code_11,APR_Medical_Surgical_Description_Surgical,APR_Medical_Surgical_Description_Medical,Payment_Typology_1_Medicaid,Payment_Typology_1_Medicare,Payment_Typology_1_Private Health Insurance,Payment_Typology_1_Blue Cross/Blue Shield,Payment_Typology_1_Self-Pay,"Payment_Typology_1_Managed Care, Unspecified",Payment_Typology_1_Federal/State/Local/VA,Payment_Typology_1_Miscellaneous/Other,Payment_Typology_1_Unknown,Payment_Typology_2_Medicare,Payment_Typology_2_Medicaid,Payment_Typology_2_Self-Pay,Payment_Typology_2_Private Health Insurance,Payment_Typology_2_Blue Cross/Blue Shield,Payment_Typology_2_Miscellaneous/Other,"Payment_Typology_2_Managed Care, Unspecified",Payment_Typology_2_Federal/State/Local/VA,Payment_Typology_3_Medicaid,Payment_Typology_3_Self-Pay,Payment_Typology_3_Blue Cross/Blue Shield,Payment_Typology_3_Private Health Insurance,Payment_Typology_3_Medicare,Emergency_Department_Indicator_N,Emergency_Department_Indicator_Y
0,0,20601.51,2,4,3,6,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,1,12020.89,1,1,5,4,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,0
2,2,40220.44,2,1,4,9,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,

In [40]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [41]:
df_pat = df_pat.drop(["Payment_Typology 1","Payment_Typology_2", "Payment_Typology_3" ] ,axis='columns')

KeyError: "['Payment_Typology 1' 'Payment_Typology_2' 'Payment_Typology_3'] not found in axis"

In [29]:
y = df_pat.Length_of_Stay
x = df_pat.drop("Length_of_Stay", axis="columns")

In [30]:
regression_model = LinearRegression()

In [31]:
regression_model.fit(x, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Function for regression training

In [23]:
def train_LR(X_train, X_test, y_train, y_test):
    # create training and testing vars
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    lm = LinearRegression()
    model = lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Score:", MSE(y_pred, y_test), '\n')
    #print(confusion_matrix(y_test, y_pred))
    #print(classification_report(y_test, y_pred))

In [27]:
y = df_pat.Length_of_Stay
X = df_pat.drop("Length_of_Stay", axis="columns")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
train_LR(X_train, X_test, y_train, y_test)

(1290, 121) (1290,)
(430, 121) (430,)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [16]:
def train_DT(X_train, X_test, y_train, y_test, i):
    # create training and testing vars
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)
    dt = DecisionTreeRegressor()
    model = dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    print("Score:", MSE(y_pred, y_test), '\n')
    #print(confusion_matrix(y_test, y_pred))
    #print(classification_report(y_test, y_pred))

In [17]:
def trainModel(df_clus, df_pat, size):
    df_clus = df_clus.transpose()
    
    # get the clusters
    clusters = []
    for i in range(size):
        this_cluster = df_clus[df_clus[i] == 1]
        clusters.append(this_cluster)
    
    # get the index (SubjectID)
    cluster = []
    for i in range(size):
        this_cluster = list(map(lambda x: int(x[1:]) - 1, list(clusters[i].index)))
        cluster.append(this_cluster)
        
    # split the patient dataframe
    pat_clustered = []
    for i in range(size):
        this_clu = df_pat[df_pat.index.isin(cluster[i])]
        pat_clustered.append(this_clu)
    for i in range(size):
        this_c = pat_clustered[i]
        if (len(this_c) > 0):
            this_df = this_c.iloc[:, 1:] #this_c.iloc[:, 2:] for 
            y = this_df.LOS
            X = this_df.iloc[:, 1:]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
            print('\n Cluster '+ str(i+1)+ ': \n')
            print('Linear Regression: ')
            train_LR(X_train, X_test, y_train, y_test, i)
            print('Decision Tree: ')
            train_DT(X_train, X_test, y_train, y_test, i)
        else:
            print('Cluster'+ str(i+1)+ ' is empty. \n')

In [ ]:
train_LR